In [1]:
# just create database in pgadmin name "chicago_crime_db" and run all programs 

import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import time
from sodapy import Socrata

In [2]:
#define json file
json_file = "https://data.cityofchicago.org/resource/kn9c-c2s2.json"
chicago_socialecnom_df = pd.read_json(json_file)
#resourses https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2


In [65]:
#chicago_socialecnom_df.dtypes

In [66]:
chicago_socialecnom_df.head()

,ca,community_area_name,hardship_index,per_capita_income_,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,percent_households_below_poverty,percent_of_housing_crowded
0,1.0,Rogers Park,39.0,23939,8.7,18.2,27.5,23.6,7.7
1,2.0,West Ridge,46.0,23040,8.8,20.8,38.5,17.2,7.8
2,3.0,Uptown,20.0,35787,8.9,11.8,22.2,24.0,3.8
3,4.0,Lincoln Square,17.0,37524,8.2,13.4,25.5,10.9,3.4
4,5.0,North Center,6.0,57123,5.2,4.5,26.2,7.5,0.3


In [6]:
#selection of attribute from main table
new_chicago_soc_df = chicago_socialecnom_df[['ca','community_area_name','per_capita_income_','percent_aged_16_unemployed','percent_aged_25_without_high_school_diploma','percent_aged_under_18_or_over_64','percent_households_below_poverty','percent_of_housing_crowded']].copy()
new_chicago_soc_df.head()


,ca,community_area_name,per_capita_income_,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,percent_households_below_poverty,percent_of_housing_crowded
0,1.0,Rogers Park,23939,8.7,18.2,27.5,23.6,7.7
1,2.0,West Ridge,23040,8.8,20.8,38.5,17.2,7.8
2,3.0,Uptown,35787,8.9,11.8,22.2,24.0,3.8
3,4.0,Lincoln Square,37524,8.2,13.4,25.5,10.9,3.4
4,5.0,North Center,57123,5.2,4.5,26.2,7.5,0.3


In [7]:
#transform dataframe
transf_chic_soc = ['ca','community_area_name','per_capita_income_','percent_aged_16_unemployed','percent_aged_25_without_high_school_diploma','percent_aged_under_18_or_over_64','percent_households_below_poverty','percent_of_housing_crowded']
transfer_chicago_spc = new_chicago_soc_df[transf_chic_soc].copy()

In [8]:
#rename the column header
transfer_chicago_spc = transfer_chicago_spc.rename(columns={"ca":"ward_no",
                                                           "community_area_name":"Areaname",
                                                           "per_capita_income_":"income_per_capital",
                                                           "percent_aged_16_unemployed":"sixteen_plus_unemployed",
                                                           "percent_aged_25_without_high_school_diploma":"twenty_plus_without_diploma",
                                                           "percent_aged_under_18_or_over_64":"population_eighteen_sixtyfour",
                                                           "percent_households_below_poverty":"population_below",
                                                           "percent_of_housing_crowded":"overpopulation"})
transfer_chicago_spc.head()
#table name in data base is "chicago_economy"

,ward_no,Areaname,income_per_capital,sixteen_plus_unemployed,twenty_plus_without_diploma,population_eighteen_sixtyfour,population_below,overpopulation
0,1.0,Rogers Park,23939,8.7,18.2,27.5,23.6,7.7
1,2.0,West Ridge,23040,8.8,20.8,38.5,17.2,7.8
2,3.0,Uptown,35787,8.9,11.8,22.2,24.0,3.8
3,4.0,Lincoln Square,37524,8.2,13.4,25.5,10.9,3.4
4,5.0,North Center,57123,5.2,4.5,26.2,7.5,0.3


In [67]:
#transfer_chicago_spc.dtypes

In [10]:
area_df = ['ca','community_area_name']
area_code_df = chicago_socialecnom_df[area_df].copy()

In [11]:
#rename the column header
area_code_df = area_code_df.rename(columns={"ca":"ward_no",
                                    "community_area_name":"Areaname"})
area_code_df.head()


,ward_no,Areaname
0,1.0,Rogers Park
1,2.0,West Ridge
2,3.0,Uptown
3,4.0,Lincoln Square
4,5.0,North Center


In [70]:
#json file
crime_df = pd.DataFrame()
for i in range(40):
    offset = 2000*i
    limit = 2000
    df = pd.read_json(f"https://data.cityofchicago.org/resource/crimes.json?$limit=2000&$offset={offset}")
    crime_df = crime_df.append(other=df)

In [71]:
crime_df.head()

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_d3ds_rm58,:@computed_region_d9mm_jgwp,:@computed_region_rpca_8um6,:@computed_region_vrxf_vc4k,arrest,beat,...,latitude,location,location_description,longitude,primary_type,updated_on,ward,x_coordinate,y_coordinate,year
0,5.0,21853.0,3.0,357.0,32.0,11.0,9.0,10.0,True,2422,...,42.019360,"{'latitude': '42.019360381', 'longitude': '-87...",SIDEWALK,-87.668783,NARCOTICS,2019-08-12T16:17:08.000,49.0,1164829.0,1950346.0,2019
1,9.0,14924.0,12.0,218.0,111.0,24.0,37.0,7.0,False,225,...,41.801285,"{'latitude': '41.801285426', 'longitude': '-87...",STREET,-87.627965,BATTERY,2019-08-12T16:17:08.000,3.0,1176574.0,1870968.0,2019
2,8.0,22268.0,35.0,173.0,112.0,13.0,7.0,53.0,False,814,...,41.798570,"{'latitude': '41.798569683', 'longitude': '-87...",CONVENIENCE STORE,-87.743534,THEFT,2019-08-12T16:17:08.000,14.0,1145070.0,1869741.0,2019
3,43.0,21861.0,30.0,9.0,221.0,10.0,47.0,51.0,False,533,...,41.659193,"{'latitude': '41.65919273', 'longitude': '-87....",STREET,-87.597698,THEFT,2019-08-12T16:17:08.000,9.0,1185269.0,1819260.0,2019
4,35.0,22538.0,9.0,188.0,222.0,19.0,24.0,39.0,False,414,...,41.756195,"{'latitude': '41.756195017', 'longitude': '-87...",RESIDENCE,-87.580975,OTHER OFFENSE,2019-08-12T16:17:08.000,8.0,1189530.0,1854646.0,2019


In [72]:
crime_df.shape

(80000, 30)

In [73]:
new_crime_df = crime_df[['ward','primary_type','arrest','location_description','year']].copy()
new_crime_df.head()

,ward,primary_type,arrest,location_description,year
0,49.0,NARCOTICS,True,SIDEWALK,2019
1,3.0,BATTERY,False,STREET,2019
2,14.0,THEFT,False,CONVENIENCE STORE,2019
3,9.0,THEFT,False,STREET,2019
4,8.0,OTHER OFFENSE,False,RESIDENCE,2019


In [68]:
#new_crime_df.shape

In [74]:
#transform dataframe
transf_crime_df = ['ward','primary_type','arrest','year','location_description']
transf_crime_chicago = new_crime_df[transf_crime_df].copy()

In [75]:
# Rename the column headers
transf_crime_chicago = transf_crime_chicago.rename(columns={"ward": "ward_no",
                                                          "primary_type": "Crime type",
                                                          "arrest": "Arrest",
                                                           "year":"Year",
                                                            "location_description":"Crime location"})

In [60]:
transf_crime_chicago.head()

,ward_no,Crime type,Arrest,Year,Crime location
0,49.0,NARCOTICS,True,2019,SIDEWALK
1,3.0,BATTERY,False,2019,STREET
2,14.0,THEFT,False,2019,CONVENIENCE STORE
3,9.0,THEFT,False,2019,STREET
4,8.0,OTHER OFFENSE,False,2019,RESIDENCE


In [14]:
### Create database connection
rds_connection_string = f"postgres:sherkhan@localhost:5432/chicago_crime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
#inserting and creating table in database
transfer_chicago_spc.to_sql(name='chicago_economy', con=engine, if_exists='append', index=True)


In [43]:
#reading data from database
pd.read_sql_query('select * from chicago_economy', con=engine).head()

,index,ward_no,Areaname,income_per_capital,sixteen_plus_unemployed,twenty_plus_without_diploma,population_eighteen_sixtyfour,population_below,overpopulation
0,0,1.0,Rogers Park,23939,8.7,18.2,27.5,23.6,7.7
1,1,2.0,West Ridge,23040,8.8,20.8,38.5,17.2,7.8
2,2,3.0,Uptown,35787,8.9,11.8,22.2,24.0,3.8
3,3,4.0,Lincoln Square,37524,8.2,13.4,25.5,10.9,3.4
4,4,5.0,North Center,57123,5.2,4.5,26.2,7.5,0.3


In [ ]:
#inserting and creating table in database
area_code_df.to_sql(name='area', con=engine, if_exists='append', index=False)

In [44]:
#reading data from database
pd.read_sql_query('select * from area', con=engine).head()

,ward_no,Areaname
0,1.0,Rogers Park
1,2.0,West Ridge
2,3.0,Uptown
3,4.0,Lincoln Square
4,5.0,North Center


In [63]:
#inserting and creating table in database
transf_crime_chicago.to_sql(name='chicago_crime', con=engine, if_exists='append', index=False)

In [64]:
#reading data from database
pd.read_sql_query('select * from chicago_crime', con=engine).head()

,ward_no,Crime type,Arrest,Year,Crime location
0,49.0,NARCOTICS,True,2019,SIDEWALK
1,3.0,BATTERY,False,2019,STREET
2,14.0,THEFT,False,2019,CONVENIENCE STORE
3,9.0,THEFT,False,2019,STREET
4,8.0,OTHER OFFENSE,False,2019,RESIDENCE
